# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
import math
from scipy import math,stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
data['peRatio']

29.8

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True
        )
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,169.360,56.25,N/A
1,AAL,19.365,-1.91,N/A
2,AAP,217.420,23.5,N/A
3,AAPL,152.050,29.74,N/A
4,ABBV,123.400,32.67,N/A
5,ABC,122.820,-6.68,N/A
6,ABMD,352.400,107.68,N/A
7,ABT,130.270,36.06,N/A
8,ACN,340.880,38.4,N/A
9,ADBE,679.410,57,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_df.sort_values('Price-to-Earnings Ratio', inplace = True)
final_df = final_df[final_df['Price-to-Earnings Ratio'] > 0]
final_df = final_df[0:50]
final_df.reset_index(drop=True, inplace=True)
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,73.96,4,N/A
1,MPC,57.48,4.83,N/A
2,ZION,54.10,5.46,N/A
3,GM,50.80,5.77,N/A
4,PRU,106.64,6.01,N/A
5,BIO,806.86,6.08,N/A
6,COF,173.47,7.03,N/A
7,AFL,58.99,7.06,N/A
8,CE,158.83,7.14,N/A
9,CINF,129.45,7.3,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size)/len(final_df.index)

for i in range(0, len(final_df.index)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Price'])
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,73.96,4,270
1,MPC,57.48,4.83,347
2,ZION,54.10,5.46,369
3,GM,50.80,5.77,393
4,PRU,106.64,6.01,187
5,BIO,806.86,6.08,24
6,COF,173.47,7.03,115
7,AFL,58.99,7.06,339
8,CE,158.83,7.14,125
9,CINF,129.45,7.3,154


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# Price-to-earnings ratio

pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio

pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio

ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)

ev = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = ev/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)

gp = data[symbol]['advanced-stats']['grossProfit']
ev_to_gp = ev/gp

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_cols = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV-Score'
]

rv_df = pd.DataFrame(columns = rv_cols)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        ev = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gp = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = ev/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gp = ev/gp
        except TypeError:
            ev_to_gp = np.NaN
        
        
        rv_df = rv_df.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['advanced-stats']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gp,
                'N/A',
                'N/A'
                
            ],
            index = rv_cols),
            ignore_index = True #calculates the index of added rows automatically
        )

In [13]:
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV-Score
0,A,175.840,N/A,57.5824,N/A,10.87,N/A,9.14,N/A,36.028629,N/A,16.301945,N/A,N/A
1,AAL,18.611,N/A,-2.13089,N/A,-1.64,N/A,0.655,N/A,-4.345126,N/A,1.807150,N/A,N/A
2,AAP,215.490,N/A,21.6792,N/A,4.03,N/A,1.29,N/A,11.610672,N/A,2.819418,N/A,N/A
3,AAPL,151.020,N/A,28.154,N/A,39.49,N/A,7.16,N/A,22.322889,N/A,17.367185,N/A,N/A
4,ABBV,120.360,N/A,31.8879,N/A,17.38,N/A,4.02,N/A,11.103973,N/A,7.914281,N/A,N/A
5,ABC,119.930,N/A,-6.9119,N/A,678.67,N/A,0.1253,N/A,9.184273,N/A,4.786555,N/A,N/A
6,ABMD,358.700,N/A,98.1799,N/A,11.58,N/A,16.96,N/A,50.428143,N/A,19.415312,N/A,N/A
7,ABT,127.380,N/A,35.5782,N/A,6.89,N/A,5.8,N/A,24.106507,N/A,10.101588,N/A,N/A
8,ACN,344.440,N/A,38.3571,N/A,11.67,N/A,4.78,N/A,23.716844,N/A,13.943815,N/A,N/A
9,ADBE,667.900,N/A,54.7686,N/A,22.23,N/A,21.56,N/A,58.415436,N/A,23.923634,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV-Score
28,ALXN,183.10,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,285.34,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,288.64,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,27.19,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.23,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,59.09,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,35.00,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2896.35,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,15.88,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
for col in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA', 'EV/GP']:
    rv_df[col].fillna(value = rv_df[col].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [16]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV-Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
metrics = {
    'Price-to-Earnings Ratio' : 'PE Percentile',
    'Price-to-Book Ratio' : 'PB Percentile',
    'Price-to-Sales Ratio' : 'PS Percentile',
    'EV/EBITDA' : 'EV/EBITDA Percentile',
    'EV/GP' : 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row, metrics[metric]] = stats.percentileofscore(rv_df[metric], rv_df.loc[row, metric])
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV-Score
0,A,175.840,N/A,57.582369,88.5149,10.870000,80.5941,9.140000,85.3465,36.028629,90.495,16.301945,86.7327,N/A
1,AAL,18.611,N/A,-2.130887,7.72277,-1.640000,4.55446,0.655000,5.74257,-4.345126,2.77228,1.807150,7.12871,N/A
2,AAP,215.490,N/A,21.679210,40,4.030000,51.8812,1.290000,17.6238,11.610672,33.6634,2.819418,14.6535,N/A
3,AAPL,151.020,N/A,28.154022,55.6436,39.490000,95.8416,7.160000,79.4059,22.322889,71.6832,17.367185,88.9109,N/A
4,ABBV,120.360,N/A,31.887856,62.5743,17.380000,90.099,4.020000,57.3267,11.103973,30.6931,7.914281,50.099,N/A
5,ABC,119.930,N/A,-6.911901,6.53465,678.670000,100,0.125300,0.594059,9.184273,22.9703,4.786555,27.3267,N/A
6,ABMD,358.700,N/A,98.179896,96.6337,11.580000,81.9802,16.960000,95.0495,50.428143,94.8515,19.415312,91.2871,N/A
7,ABT,127.380,N/A,35.578237,69.1089,6.890000,68.9109,5.800000,68.9109,24.106507,75.4455,10.101588,65.9406,N/A
8,ACN,344.440,N/A,38.357066,74.2574,11.670000,82.5743,4.780000,62.7723,23.716844,74.4554,13.943815,80.5941,N/A
9,ADBE,667.900,N/A,54.768645,86.7327,22.230000,92.5743,21.560000,97.6238,58.415436,96.8317,23.923634,95.8416,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [18]:
from statistics import mean

for row in rv_df.index:
    pe_ratio = rv_df.loc[row, 'PE Percentile']
    pb_ratio = rv_df.loc[row, 'PB Percentile']
    ps_ratio = rv_df.loc[row, 'PS Percentile']
    eveb_ratio = rv_df.loc[row, 'EV/EBITDA Percentile']
    evgp_ratio = rv_df.loc[row, 'EV/GP Percentile']
    rv_df.loc[row, 'RV-Score'] = mean([pe_ratio, pb_ratio, ps_ratio, eveb_ratio, evgp_ratio])
    # alternate method to do same but without repeating code
#     value_perc = []
#     for metric in metrics.keys():
#         value_perc.append(rv_df.loc[row, metrics[metric]])
#     rv_df.loc[row, 'RV-Score'] = mean(value_perc)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV-Score
0,A,175.840,N/A,57.582369,88.5149,10.870000,80.5941,9.140000,85.3465,36.028629,90.495,16.301945,86.7327,86.3366
1,AAL,18.611,N/A,-2.130887,7.72277,-1.640000,4.55446,0.655000,5.74257,-4.345126,2.77228,1.807150,7.12871,5.58416
2,AAP,215.490,N/A,21.679210,40,4.030000,51.8812,1.290000,17.6238,11.610672,33.6634,2.819418,14.6535,31.5644
3,AAPL,151.020,N/A,28.154022,55.6436,39.490000,95.8416,7.160000,79.4059,22.322889,71.6832,17.367185,88.9109,78.297
4,ABBV,120.360,N/A,31.887856,62.5743,17.380000,90.099,4.020000,57.3267,11.103973,30.6931,7.914281,50.099,58.1584
5,ABC,119.930,N/A,-6.911901,6.53465,678.670000,100,0.125300,0.594059,9.184273,22.9703,4.786555,27.3267,31.4851
6,ABMD,358.700,N/A,98.179896,96.6337,11.580000,81.9802,16.960000,95.0495,50.428143,94.8515,19.415312,91.2871,91.9604
7,ABT,127.380,N/A,35.578237,69.1089,6.890000,68.9109,5.800000,68.9109,24.106507,75.4455,10.101588,65.9406,69.6634
8,ACN,344.440,N/A,38.357066,74.2574,11.670000,82.5743,4.780000,62.7723,23.716844,74.4554,13.943815,80.5941,74.9307
9,ADBE,667.900,N/A,54.768645,86.7327,22.230000,92.5743,21.560000,97.6238,58.415436,96.8317,23.923634,95.8416,93.9208


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [24]:
rv_df.sort_values('RV-Score', ascending = True, inplace = True) #50-cheapest stocks
rv_df = rv_df[:50]
rv_df.reset_index(drop = True, inplace = True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV-Score
0,UNM,26.060,N/A,7.359247,10.8911,0.4806,4.75248,0.3933,2.57426,3.366479,3.9604,0.394986,0.990099,4.63366
1,PRU,108.970,N/A,5.680427,8.91089,0.6398,5.14851,0.6301,5.14851,4.750407,5.14851,0.625496,1.38614,5.14851
2,AAL,18.611,N/A,-2.130887,7.72277,-1.6400,4.55446,0.6550,5.74257,-4.345126,2.77228,1.807150,7.12871,5.58416
3,AIV,6.840,N/A,-19.458494,2.57426,1.9600,26.6337,-4.5700,0.19802,-13.795219,1.58416,-10.059517,0.792079,6.35644
4,L,56.600,N/A,9.345890,14.6535,0.8029,6.73267,0.9578,9.50495,5.232986,6.13861,0.928722,2.37624,7.88119
5,LNC,66.880,N/A,8.919228,13.8614,0.5711,4.9505,0.6393,5.34653,7.146081,13.8614,0.640196,1.58416,7.92079
6,MCK,209.860,N/A,-7.124254,6.13861,-58.9100,0.990099,0.1315,0.792079,8.128945,18.2178,2.775841,13.8614,8
7,GM,49.600,N/A,5.865623,9.30693,1.3800,14.7525,0.5350,3.9604,2.168940,3.56436,1.949627,8.71287,8.05941
8,BA,216.830,N/A,-14.724519,4.55446,-7.4500,3.76238,2.0500,31.0891,-30.144928,0.792079,-208.206984,0.19802,8.07921
9,MET,61.730,N/A,12.097480,20.9901,0.7631,5.94059,0.7476,7.12871,5.752482,7.52475,0.737626,1.78218,8.67327


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [25]:
portfolio_input()

Enter the value of your portfolio:10000000


In [33]:
position_size = float(portfolio_size)/len(rv_df.index)

for row in rv_df.index:
    rv_df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_df.loc[row, 'Price'])

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [55]:
writer = pd.ExcelWriter('Value_strategy.xlsx', engine='xlsxwriter')
rv_df.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [56]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [57]:
column_format = {
    'A' : ['Ticker', string_template],
    'B' : ['Price',dollar_template],
    'C' : ['Number of Shares to Buy', integer_template],
    'D' : ['Price-to-Earnings Ratio', float_template],
    'E' : ['PE Percentile', percent_template],
    'F' : ['Price-to-Book Ratio', float_template],
    'G' : ['PB Percentile', percent_template],
    'H' : ['Price-to-Sales Ratio', float_template],
    'I' : ['PS Percentile', percent_template],
    'J' : ['EV/EBITDA', float_template],
    'K' : ['EV/EBITDA Percentile', percent_template],
    'L' : ['EV/GP', float_template],
    'M' : ['EV/GP Percentile', percent_template],
    'N' : ['RV-Score', integer_template]
}

for col in column_format.keys():
    writer.sheets['Value Strategy'].set_column(f'{col}:{col}', 25, column_format[col][1])
    writer.sheets['Value Strategy'].write(f'{col}1', column_format[col][0], column_format[col][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [58]:
writer.save()